In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# %sql
# SELECT * FROM workspace.silver.silver_flights

# Paramaters for Flights

In [0]:
#catalog
catalog = "workspace"

# Key Cols List
key_cols = "['flight_id']"
key_cols_list = eval(key_cols)

#CDC Column
cdc_col = "modifiedDate"

#Back dated Refresh
backdated_refresh = ""

#Source Object
source_object = "silver_flights"

#Source Schema
source_schema = "silver"

#Target Object
target_object = "DimFlights"

#Target Schema
target_schema = "gold"

#Surrogate Key Column
surrogate_key = "DimFlightsKey"

# Paramaters for Airports

In [0]:
# #catalog
# catalog = "workspace"

# # Key Cols List
# key_cols = "['airport_id']"
# key_cols_list = eval(key_cols)

# #CDC Column
# cdc_col = "modifiedDate"

# #Back dated Refresh
# backdated_refresh = ""

# #Source Object
# source_object = "silver_airports"

# #Source Schema
# source_schema = "silver"

# #Target Object
# target_object = "DimAirports"

# #Target Schema
# target_schema = "gold"

# #Surrogate Key Column
# surrogate_key = "DimAirportsKey"

# Paramaters for Parameters

In [0]:
# #catalog
# catalog = "workspace"

# # Key Cols List
# key_cols = "['passenger_id']"
# key_cols_list = eval(key_cols)

# #CDC Column
# cdc_col = "modifiedDate"

# #Back dated Refresh
# backdated_refresh = ""

# #Source Object
# source_object = "silver_passengers"

# #Source Schema
# source_schema = "silver"

# #Target Object
# target_object = "DimPassengers"

# #Target Schema
# target_schema = "gold"

# #Surrogate Key Column
# surrogate_key = "DimPassengersKey"

# **Incremental Data Ingestion**

**Last Load Date**

In [0]:
#No Back Dated Refresh
if len(backdated_refresh) == 0:
    
    #If table exists in the destination 
    if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

        last_load = spark.sql(f"SELECT max({cdc_col}) FROM {catalog}.{target_schema}.{target_object}").collect()[0][0]
    
    else:
        
        last_load = "1900-01-01 00:00:00"

#Back Dated Refresh
else:

    last_load = backdated_refresh

last_load

datetime.datetime(2025, 7, 14, 7, 0, 42, 52000)

In [0]:
df_src = spark.sql(f"SELECT * FROM {catalog}.{source_schema}.{source_object} WHERE {cdc_col} > '{last_load}'")

In [0]:
#df_src.display()

flight_id,airline,origin,destination,flight_date,modifiedDate
F0113,Air Asia,North Mistyberg,East Lisaton,2025-07-02,2025-07-15T14:22:01.525Z
F0111,United,South Michellemouth,Jacquelinechester,2025-07-12,2025-07-15T14:22:01.525Z
F0112,Ryanair,Lake David,Port Michael,2025-07-12,2025-07-15T14:22:01.525Z
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-15T14:22:01.525Z
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-15T14:22:01.525Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-15T14:22:01.525Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-15T14:22:01.525Z
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-15T14:22:01.525Z


**OLD vs NEW Records**

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    #Key Columns String for Incremental
    key_cols_string_incremental = ", ".join(key_cols_list)
    
    df_trg = spark.sql(f"SELECT {key_cols_string_incremental}, {surrogate_key}, create_date, update_date  FROM {catalog}.{target_schema}.{target_object}")

else: 

    #Key Columns String for Initial
    key_cols_string_init = [f"'' AS {i} " for i in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)
    df_trg = spark.sql(f"""SELECT {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key}, CAST('1900-01-01 00:00:00' as timestamp) AS create_date, 
                                                CAST('1900-01-01 00:00:00' as timestamp) AS update_date  WHERE 1=0 """)


In [0]:
#df_trg.display()

### Join condition

In [0]:
join_condition = ' AND '.join([f"src.{i} = trg.{i}" for i in key_cols_list])

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""
SELECT
    src.*,
    trg.{surrogate_key},
    trg.create_date,
    trg.update_date
FROM
    src
    LEFT JOIN trg
        ON {join_condition}
""")

In [0]:
#df_join.display()

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0113,Air Asia,North Mistyberg,East Lisaton,2025-07-02,2025-07-15T14:22:01.525Z,null,null,null
F0111,United,South Michellemouth,Jacquelinechester,2025-07-12,2025-07-15T14:22:01.525Z,null,null,null
F0112,Ryanair,Lake David,Port Michael,2025-07-12,2025-07-15T14:22:01.525Z,null,null,null
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-15T14:22:01.525Z,13,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-15T14:22:01.525Z,4,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-15T14:22:01.525Z,40,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-15T14:22:01.525Z,7,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-15T14:22:01.525Z,16,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z


In [0]:
#OLD Records
df_old = df_join.filter(col(f'{surrogate_key}').isNotNull())

#NEW Records
df_new = df_join.filter(col(f'{surrogate_key}').isNull()) 

#df_old.display()
#df_new.display()

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-15T14:22:01.525Z,13,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-15T14:22:01.525Z,4,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-15T14:22:01.525Z,40,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-15T14:22:01.525Z,7,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-15T14:22:01.525Z,16,2025-07-14T17:55:47.077Z,2025-07-14T18:09:38.512Z


flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0112,Ryanair,Lake David,Port Michael,2025-07-12,2025-07-15T14:22:01.525Z,null,null,null
F0111,United,South Michellemouth,Jacquelinechester,2025-07-12,2025-07-15T14:22:01.525Z,null,null,null
F0113,Air Asia,North Mistyberg,East Lisaton,2025-07-02,2025-07-15T14:22:01.525Z,null,null,null


### Preparing df_old

In [0]:
df_old_enr = df_old.withColumn('update_date',current_timestamp())

### Preparing df_new

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
  max_surrogate_key = spark.sql(f"""
                                SELECT max({surrogate_key}) FROM {catalog}.{target_schema}.{target_object}
                                """).collect()[0][0]
  df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
                  .withColumn('create_date',current_timestamp())\
                  .withColumn('update_date',current_timestamp())

else:
  max_surrogate_key = 0
  df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key)+lit(1)+monotonically_increasing_id())\
                  .withColumn('create_date',current_timestamp())\
                  .withColumn('update_date',current_timestamp())

In [0]:
df_old_enr.display()

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-15T14:22:01.525Z,13,2025-07-14T17:55:47.077Z,2025-07-15T14:31:18.562Z
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-15T14:22:01.525Z,4,2025-07-14T17:55:47.077Z,2025-07-15T14:31:18.562Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-15T14:22:01.525Z,40,2025-07-14T17:55:47.077Z,2025-07-15T14:31:18.562Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-15T14:22:01.525Z,7,2025-07-14T17:55:47.077Z,2025-07-15T14:31:18.562Z
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-15T14:22:01.525Z,16,2025-07-14T17:55:47.077Z,2025-07-15T14:31:18.562Z


### Union of OLD and NEW records 

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)
df_union.display()

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-15T14:22:01.525Z,13,2025-07-14T17:55:47.077Z,2025-07-15T14:31:19.635Z
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-15T14:22:01.525Z,4,2025-07-14T17:55:47.077Z,2025-07-15T14:31:19.635Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-15T14:22:01.525Z,40,2025-07-14T17:55:47.077Z,2025-07-15T14:31:19.635Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-15T14:22:01.525Z,7,2025-07-14T17:55:47.077Z,2025-07-15T14:31:19.635Z
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-15T14:22:01.525Z,16,2025-07-14T17:55:47.077Z,2025-07-15T14:31:19.635Z
F0112,Ryanair,Lake David,Port Michael,2025-07-12,2025-07-15T14:22:01.525Z,111,2025-07-15T14:31:19.635Z,2025-07-15T14:31:19.635Z
F0111,United,South Michellemouth,Jacquelinechester,2025-07-12,2025-07-15T14:22:01.525Z,112,2025-07-15T14:31:19.635Z,2025-07-15T14:31:19.635Z
F0113,Air Asia,North Mistyberg,East Lisaton,2025-07-02,2025-07-15T14:22:01.525Z,113,2025-07-15T14:31:19.635Z,2025-07-15T14:31:19.635Z


### Upsert

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
  dlt_obj = DeltaTable.forName(spark,f"{catalog}.{target_schema}.{target_object}")
  dlt_obj.alias("trg").merge(df_union.alias("src"),f"trg.{surrogate_key} = src.{surrogate_key}")\
                        .whenMatchedUpdateAll(condition = f'src.{cdc_col} >= trg.{cdc_col}')\
                        .whenNotMatchedInsertAll()\
                        .execute()
else:
  df_union.write.format('delta')\
                .mode("append")\
                .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
SELECT * FROM workspace.gold.dimpassengers LIMIT 10

passenger_id,name,gender,nationality,modifiedDate,DimPassengersKey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-14T18:37:51.022Z,7,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0108,Brian Anderson,Female,Eritrea,2025-07-14T18:37:51.022Z,36,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0032,Daniel Douglas,Male,Singapore,2025-07-14T18:37:51.022Z,46,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0113,Brian Richardson,Male,Dominican Republic,2025-07-14T18:37:51.022Z,50,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-14T18:37:51.022Z,53,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0158,Jamie Mcneil,Male,Burundi,2025-07-14T18:37:51.022Z,76,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0155,Robert Singleton,Female,Northern Mariana Islands,2025-07-14T18:37:51.022Z,82,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0196,Monica Dillon,Female,Belarus,2025-07-14T18:37:51.022Z,86,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0154,Angel Thompson,Female,Gambia,2025-07-14T18:37:51.022Z,110,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-14T18:37:51.022Z,197,2025-07-14T18:21:46.226Z,2025-07-14T18:41:08.649Z
